# Loss comparison (DA vs. weighted particles)
Given a regular radial scan performed with Sixtrack, we try different distributions and compare the lost amount of beam.

## Set correct working directory and install libraries in SWAN instance
(since SWAN generates a new instance of the notebook in another empty directory)

In [ ]:
# Working in the right path
%cd /eos/project/d/da-and-diffusion-studies/DA_Studies/Simulations/Models/da_sixtrack

In [ ]:
# Install the libraries
import sys
!{sys.executable} -m pip install --user tqdm pynverse
!export PYTHONPATH=$CERNBOX_HOME/.local/lib/python3.7/site-packages:$PYTHONPATH

In [ ]:
# For this "presentation" only! As some plotting parts execute a np.log10(0)
import warnings
warnings.filterwarnings('ignore')

## Import libraries

In [1]:
%matplotlib widget

In [2]:
# Base libraries
import math
import numpy as np
import scipy.integrate as integrate
from scipy.special import erf
import pickle
import itertools
from scipy.optimize import curve_fit

from numba import njit, prange

# Personal libraries
#import sixtrackwrap_light as sx
import sixtrackwrap_light as sx

from tqdm.notebook import tqdm
import time
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
import matplotlib
import matplotlib.ticker as ticker
from math import gcd

from scipy.special import lambertw

## Load data and setup

In [3]:
savepath = "data/"
engine = sx.radial_scanner.load_values(savepath + "big_scan.pkl")

min_turns = engine.min_time
max_turns = engine.max_time
n_turn_samples = 500

turn_sampling = np.linspace(min_turns, max_turns, n_turn_samples, dtype=np.int_)[::-1]

d_r = engine.dr
starting_step = engine.starting_step

# BASELINE COMPUTING
baseline_samples = 33
baseline_total_samples = baseline_samples ** 3

In [4]:
alpha_preliminary_values = np.linspace(-1.0, 1.0, baseline_samples)
alpha_values = np.arccos(alpha_preliminary_values) / 2
theta1_values = np.linspace(0.0, np.pi * 2.0, baseline_samples, endpoint=False)
theta2_values = np.linspace(0.0, np.pi * 2.0, baseline_samples, endpoint=False)

d_preliminar_alpha = alpha_preliminary_values[1] - alpha_preliminary_values[0]
d_theta1 = theta1_values[1] - theta1_values[0]
d_theta2 = theta2_values[1] - theta2_values[0]

alpha_mesh, theta1_mesh, theta2_mesh = np.meshgrid(alpha_values, theta1_values, theta2_values, indexing='ij')

alpha_flat = alpha_mesh.flatten()
theta1_flat = theta1_mesh.flatten()
theta2_flat = theta2_mesh.flatten()

## A Colormap for roughly visualize all the samples (regardless of the angle)
Here we just observe all the gathered radial samples "as a bunch".

I choose a logarithmic visualization of the stability value in order to better visualize the variation of the number of stable turns (white means absence of data, i.e. the particle was not simulated).

In [5]:
fig1, ax1 = plt.subplots()
cmap = ax1.imshow(np.log10(engine.steps), aspect="auto", extent=(engine.starting_step, engine.starting_step + engine.dr * len(engine.steps[0]),0,len(engine.steps)))
ax1.set_ylabel("Radial sample number")
ax1.set_xlabel("Radial distance [normalized Sigma units]")
cbar = plt.colorbar(cmap)
cbar.ax.set_ylabel("Number of stable turns $\\left[\\log_{10}(N_{turns})\\right]$")
ax1.set_title("Heatmap view of the radial scans")
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/carlidel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log10
  


## Exploring and visualizing 3D samples of DA!

With this tool, you can (somewhat) visualize the angular dependencies of DA by moving the $\theta_1$ and $\theta_2$ sliders and setting up 3D samples of different dimension (the resulting sample is sample_size ** 3 big).

What you will then visualize is the evolution of DA with the number of turns, considering different $\alpha$ angles ($\alpha$ indicates the central angle of the considered sample).

**N.B.: the plotting process requires time, so after moving the sliders you will need to wait a little!**

In [6]:
fig, ax = plt.subplots()
cmap = matplotlib.cm.get_cmap('viridis')
norm = matplotlib.colors.Normalize(vmin=np.log10(turn_sampling[-1]), vmax=np.log10(turn_sampling[0]))
fig.colorbar(matplotlib.cm.ScalarMappable(norm=norm, cmap=cmap), label='Number of stable turns considered\n$[\\log_{10}(N_{turns})]$')

radiuses = engine.extract_DA(turn_sampling)
radiuses = radiuses.reshape((baseline_samples, baseline_samples, baseline_samples, len(turn_sampling)))

@njit
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

@njit
def take_sample(array, value, size):
    assert size % 2 == 0
    array = np.asarray(array)
    idx = find_nearest(array, value)
    if idx < size:
        return 0, size
    elif idx >= len(array) - size:
        return len(array) - size, len(array)
    else:
        return idx - size // 2, idx + size // 2

def update1(sample_size, th1, th2):
    th1 *= np.pi
    th2 *= np.pi
    y_values = np.empty((len(range(sample_size, len(alpha_preliminary_values))), len(turn_sampling)))
    x_values = np.empty((len(range(sample_size, len(alpha_preliminary_values)))))
    for i, a_max in enumerate(range(sample_size, len(alpha_preliminary_values))):
        a_min = a_max - sample_size
        
        th1_min, th1_max = take_sample(theta1_values, th1, sample_size)
        th2_min, th2_max = take_sample(theta2_values, th2, sample_size)
        alpha_sample = alpha_preliminary_values[a_min : a_max]
        theta1_sample = theta1_values[th1_min : th1_max]
        theta2_sample = theta1_values[th2_min : th2_max]

        a_mid = (alpha_values[a_min] + alpha_values[a_max]) / 2
        
        mod_radiuses = np.power(radiuses, 4)[a_min : a_max, th1_min : th1_max, th2_min : th2_max]
        
        mod_radiuses = integrate.simps(mod_radiuses, x=theta1_sample, axis=1)
        mod_radiuses = integrate.simps(mod_radiuses, x=theta2_sample, axis=1)
        mod_radiuses = integrate.simps(mod_radiuses, x=alpha_sample, axis=0)

        DA = (
            np.power(
                mod_radiuses / (
                    (alpha_sample[-1] - alpha_sample[0]) 
                    * (theta1_sample[-1] - theta1_sample[0]) 
                    * (theta2_sample[-1] - theta2_sample[0])),
                1/4
            )
        )
        y_values[i] = DA
        x_values[i] = a_mid
    y_values = np.asarray(y_values)
    y_values = y_values.transpose()
    x_values = np.asarray(x_values)
    ax.clear()
    for i in range(y_values.shape[0]):
        value = np.log10(turn_sampling[i] - turn_sampling[-1]) / np.log10(turn_sampling[0] - turn_sampling[-1])
        ax.plot(x_values, y_values[i], c=cmap(value))
    ax.set_xlabel("Central $\\alpha$ angle")
    ax.set_ylabel("Measured $DA$ in sample")
    ax.set_title("DA evolution over $\\alpha$ for a moving average of ${}^3$ elements (total is ${}^3$)\nCentral $\\theta$ angles: $(\\theta_1 = {:.2f}\\pi, \\theta_2 = {:.2f}\\pi)$".format(sample_size, baseline_samples, th1/np.pi, th2/np.pi, baseline_samples))
    ax.set_ylim(np.min(radiuses), np.max(radiuses))
    ax.set_xlim(0.0, np.pi / 2.0)
    ax.xaxis.set_major_formatter(
        ticker.FuncFormatter(
            lambda x, pos: ("$\\frac{{{}}}{{{}}}$".format(int(x/(np.pi/8)) // gcd(8, int(x/(np.pi/8))), 8 // gcd(8, int(x/(np.pi/8)))) if x != 0 else "0") + "$\\pi$"
        )
    )
    ax.xaxis.set_major_locator(ticker.MultipleLocator(base=np.pi/8))
    plt.tight_layout()


a=widgets.IntSlider(value=2, min=2, max=baseline_samples - 4, step=2, continuous_update=False)
b=widgets.FloatSlider(value=1, min=0, max=2 + 0.01, step=0.01, continuous_update=False)
c=widgets.FloatSlider(value=1, min=0, max=2 + 0.01, step=0.01, continuous_update=False)
ui = widgets.VBox([
    widgets.Label("Size of the cubic sample"), a,
    widgets.Label("$\\theta_1$ value $[\\pi$ units$]$"), b,
    widgets.Label("$\\theta_2$ value $[\\pi$ units$]$"), c])
    
out = widgets.interactive_output(
    update1,
    {"sample_size":a, "th1":b, "th2":c}
)

display(ui, out)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Output()

# Setup for loss comparison analysis

In [7]:
radiuses = engine.extract_DA(turn_sampling)
radiuses = radiuses.reshape((baseline_samples, baseline_samples, baseline_samples, len(turn_sampling)))

In [8]:
DA = []

mod_radiuses = radiuses.copy()
mod_radiuses = np.power(radiuses, 4)
mod_radiuses1 = integrate.simps(mod_radiuses, x=theta1_values, axis=1)
mod_radiuses2 = integrate.simps(mod_radiuses1, x=theta2_values, axis=1)
mod_radiuses3 = integrate.simps(mod_radiuses2, x=alpha_preliminary_values, axis=0)

for i in range(len(turn_sampling)):
    DA.append(
        np.power(
            mod_radiuses3[i] / (2 * theta1_values[-1] * theta2_values[-1]),
            1/4
        )
    )

DA = np.asarray(DA)

In [9]:
axis_sampling = np.concatenate((turn_sampling,[0.0]))

# How is the error on the DA loss computed right now?

1. Consider all the radiuses sampled.
2. Compute the DA value.
3. For every radius sampled, compute the difference from the DA value.
4. The absolute value of the average of all these differences is considered as error.

(I tried using the Standard Deviation of the radiuses distribution, but it ended up being 10% of the DA itself, so we "need" somehow a smaller error estimation)

# Uniform distribution case

In [10]:
def loss_from_DA(da_list, da_cut):
    temp = np.pi ** 2 / 2 * np.power(da_list, 4)
    return np.concatenate((temp, [np.pi ** 2 / 2 * np.power(da_cut, 4)]))

values = loss_from_DA(DA, 26.0)
values /= values[-1]
values[-20:]

# Error computing

values1 = loss_from_DA(DA - np.absolute(np.mean(radiuses - DA, axis=(0,1,2))), 26)
values1 /= values1[-1]

values2 = loss_from_DA(DA + np.absolute(np.mean(radiuses - DA, axis=(0,1,2))), 26)
values2 /= values2[-1]

In [11]:
engine.assign_weights(
    sx.assign_uniform_distribution()
)

In [12]:
real_values = engine.compute_loss(turn_sampling, 26)
real_values
fig3, ax3 = plt.subplots()
ax3.plot(axis_sampling, values, label="Values from DA")
ax3.fill_between(axis_sampling, values1, values2, label="Values from DA - error", color="C0", alpha=0.4)
ax3.plot(axis_sampling, real_values, label="Values from weights")
ax3.legend()
ax3.set_xlabel("$N$ turns")
ax3.set_ylabel("Active beam")
ax3.set_title("Uniform beam (Cutting Point at $DA=26.0$)")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Uniform beam (Cutting Point at $DA=26.0$)')

# Gaussian with slider available for $\sigma$

With the slider it is possible to regulate the $\sigma$ value of the 4D gaussian beam distribution.

It is possible to observe how extreme values for $\sigma$ strongly changes the two loss behaviours.

**N.B.: Place the slider at the desired position and then press the button!**

In [13]:
fig2, ax2 = plt.subplots()

def update2(sigma):
    ax2.clear()
    # Cursed Sanity Check
    def loss_from_DA(DA_list, DA_max):
        temp = - np.exp(- ((DA_list / sigma) ** 2) / 2) * (DA_list ** 2 + 2 * sigma ** 2) + 2 * sigma ** 2
        return np.concatenate((temp, [- np.exp(- ((DA_max / sigma) ** 2) / 2) * (DA_max ** 2 + 2 * sigma ** 2) + 2 * sigma ** 2]))

    values = loss_from_DA(DA, 26.0)
    values /= values[-1]
    values[-20:]

    # Error computing
    
    values1 = loss_from_DA(DA - np.absolute(np.mean(radiuses - DA, axis=(0,1,2))), 26)
    values1 /= values1[-1]

    values2 = loss_from_DA(DA + np.absolute(np.mean(radiuses - DA, axis=(0,1,2))), 26)
    values2 /= values2[-1]

    engine.assign_weights(
        sx.assign_symmetric_gaussian(sigma)
    )

    real_values = engine.compute_loss(turn_sampling, 26)
    ax2.plot(axis_sampling, values, label="Values from DA")
    ax2.fill_between(axis_sampling, values1, values2, color="C0", alpha=0.4)
    ax2.plot(axis_sampling, real_values, label="Values from weights")
    ax2.legend()
    ax2.set_xlabel("$N$ turns")
    ax2.set_ylabel("Active beam")
    ax2.set_title("Symmetric gaussian beam\n($\\sigma = {}$, cutting Point at $DA=26.0$)".format(sigma))
    plt.tight_layout()

widgets.interact_manual(update2, sigma=widgets.FloatSlider(value=15, min=1, max=30, step=0.5))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=15.0, description='sigma', max=30.0, min=1.0, step=0.5), Button(descri…

<function __main__.update2(sigma)>

# A contour plot for analyzing the N turns / Sigma relation in the loss difference between real values and DA-based values for a symmetric gaussian distribution

## Data computation (this takes a long time!)
Here we compute the loss for many different values of sigma

In [14]:
sigma_samples = 100
sigma_list = np.linspace(2.0, 30, sigma_samples)

r_values = np.empty((sigma_samples, len(axis_sampling)))
DA_values = np.empty((sigma_samples, len(axis_sampling)))

@njit
def loss_from_DA(DA_list, DA_max, sigma):
        temp = - np.exp(- ((DA_list / sigma) ** 2) / 2) * (DA_list ** 2 + 2 * sigma ** 2) + 2 * sigma ** 2
        temp = np.concatenate((temp, np.array([- np.exp(- ((DA_max / sigma) ** 2) / 2) * (DA_max ** 2 + 2 * sigma ** 2) + 2 * sigma ** 2])))
        return temp
    
    
for i, sigma in tqdm(enumerate(sigma_list), total=len(sigma_list)):
    values = loss_from_DA(DA, 26.0, sigma)
    values /= values[-1]
    DA_values[i] = values
    engine.assign_weights(
        sx.assign_symmetric_gaussian(sigma)
    )

    real_values = engine.compute_loss(turn_sampling, 26)
    r_values[i] = real_values

## Data visualization
### Two colormaps for basic comparison
Here we visualize how the relative loss changes depending on the $\sigma$ value of the 4D gaussian distribution

In [15]:
fig4, axs4 = plt.subplots(ncols=2)

global_min = min(np.min(r_values), np.min(DA_values))

axs4[0].imshow(r_values[:,::-1], aspect='auto', extent=(axis_sampling[-1], axis_sampling[0], sigma_list[0], sigma_list[-1]), vmin=global_min, vmax=1)
axs4[0].set_xlabel("$N$ turns")
axs4[0].set_ylabel("$\\sigma$ value")
axs4[0].set_title("Loss from actual lost particles")

im = axs4[1].imshow(DA_values[:,::-1], aspect='auto', extent=(axis_sampling[-1], axis_sampling[0], sigma_list[0], sigma_list[-1]), vmin=global_min, vmax=1)
axs4[1].set_xlabel("$N$ turns")
axs4[1].set_ylabel("$\\sigma$ value")
axs4[1].set_title("Loss from DA extrapolation")

fig4.subplots_adjust(right=0.85)
cbar_ax = fig4.add_axes([0.90, 0.15, 0.02, 0.7])
fig.colorbar(im, cax=cbar_ax, label="Relative loss")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## A contour plot for the difference evolution

Here we visualize the evolution of the relative error value:

$$ \frac{|\text{Loss}_{\text{true}}-\text{Loss}_{\text{DA}}|}{\text{Loss}_{\text{true}}}$$

**N.B.** for extremely low $\sigma$ values we degenerate to a near-zero error since we register almost no loss at all with the scanning we are working with.

In [16]:
fig5, ax5 = plt.subplots()
skip = -50
XX, YY = np.meshgrid(axis_sampling, sigma_list)
img = ax5.contourf(XX[:,:skip], YY[:,:skip], (np.absolute(DA_values - r_values)/r_values)[:,:skip], levels=10)
fig5.colorbar(img, label="Relative error")
ax5.set_xlabel("$N$ turns")
ax5.set_ylabel("$\\sigma$ value")
ax5.set_title("Relative difference between real loss and DA loss")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Relative difference between real loss and DA loss')